# Importing the necessary library files

In [1]:
#selenium libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, ElementClickInterceptedException

import time
import pandas as pd
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

import requests
import urllib.parse
import os
import shutil

## Now we would be defining a commont function which would help us to load webdriver and launch a url

In [11]:
def launch_drive(url):
    #creating driver instance
    driver = webdriver.Chrome(r"C://Users/hp/Downloads/chromedriver_win32/chromedriver.exe")
    driver.maximize_window()
    
    #defining implicit wait
    driver.implicitly_wait(10)
    
    #launching the url
    driver.get(url)
    
    return driver

### 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [12]:
url="https://www.amazon.in/"
keyword=input("Enter item to be searched\n")

Enter item to be searched
Guitars


In [13]:
## Now we are defining a function which would help us to search a product on amazon.com
def amazon_product_search(keyword):
    
    url = 'https://www.amazon.in/'
    
    drivr = launch_drive(url)
    drivr.find_element_by_id("twotabsearchtextbox").send_keys(keyword)
    drivr.find_element_by_xpath("//input[@type='submit' and @value='Go']").click()
    search_text = drivr.find_element_by_xpath("//div[@class='a-section a-spacing-small a-spacing-top-small']").text
    print("Showing",search_text)
    return drivr

In [14]:
product=amazon_product_search(keyword)

Showing 1-48 of over 1,000 results for "Guitars"


In [15]:
product.quit()

### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [20]:

def fetch_product_details_amazon(product, pages, save_as_csv = False):
    d = amazon_product_search(keyword)
    num_pages = pages
    page_num = 1
    time.sleep(2)
    brands = []
    product_names = []
    ratings = []
    num_ratings = []
    prices = []
    expected_delivery = []
    availabilities = []
    prod_urls = []
    other_details = []
    returns = []

    next_tab = True

    while (page_num < num_pages + 1):
        time.sleep(2)
        page_num += 1
        main_window = d.current_window_handle

        plp_product_elems = d.find_elements_by_xpath("//div[@data-component-type='s-search-result']")
        
        for prod_elems in range(len(plp_product_elems)):
            try:
                try:
                    plp_product_elems[prod_elems].click()
                except:
                    continue
            except StaleElementReferenceException:
                try:
                    d.get(d.current_url)
                    time.sleep(2)
                    plp_product_elems = d.find_elements_by_xpath("//div[@data-component-type='s-search-result']")
                    #d.navigate().refresh()
                    time.sleep(2)
                    plp_product_elems[prod_elems].click()
                except:
                    continue
            except ElementNotInteractableException:
                d.find_element_by_xpath("//a[@class='a-link-normal a-carousel-goto-nextpage s-carousel-button']").click()
                time.sleep(1)
                prod_elems.click()

            all_windows = d.window_handles

            if len(all_windows) > 1:
                try:
                    d.switch_to.window(all_windows[1])
                except NoSuchWindowException:
                    continue
                next_tab = True
            else:
                next_tab = False

            try:
                try:
                    try:
                        brands.append(d.find_element_by_id("bylineInfo").text)
                    except:
                        brands.append("-")
                except NoSuchElementException:
                    brands.append("-") 
                try:
                    product_names.append(d.find_element_by_id("productTitle").text.strip())
                except NoSuchElementException:
                    product_names.append("-")
                try:
                    rating = d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@class='a-icon-alt']").text.strip()
                    ratings.append(rating)
                    num_ratings.append(d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@id='acrCustomerReviewText']").text.strip())
                except NoSuchElementException:
                    ratings.append("-")
                    num_ratings.append("-")
                try:
                    prices.append(d.find_element_by_id("priceblock_ourprice").text.strip())
                except NoSuchElementException:
                    try:
                        prices.append(d.find_element_by_id("priceblock_dealprice").text.strip())
                    except NoSuchElementException:
                        try:
                            prices.append(d.find_element_by_id("priceblock_saleprice").text.strip())
                        except NoSuchElementException:
                            prices.append("-")
                try:
                    expected_delivery.append(d.find_element_by_id("ddmDeliveryMessage").text.strip())
                except NoSuchElementException:
                    expected_delivery.append("-")
                try:
                    avl = d.find_element_by_id("availability").text.strip()
                    if len(avl) > 0:
                        availabilities.append(avl)
                    else:
                        availabilities.append("-")
                except NoSuchElementException:
                    availabilities.append("-")
                prod_urls.append(d.current_url)
                try:
                    other_details.append(d.find_element_by_id('feature-bullets').text)
                except NoSuchElementException:
                    other_details.append("-")
                try:
                    returns.append(d.find_element_by_xpath("//div[@id='RETURNS_POLICY']//a").text)
                except NoSuchElementException:
                    returns.append("-")
            except StaleElementReferenceException:
                d.navigate().refresh()
                time.sleep(2)
                try:
                    brands.append(d.find_element_by_id("bylineInfo").text)
                except NoSuchElementException:
                    brands.append("-")
                try:
                    product_names.append(d.find_element_by_id("productTitle").text.strip())
                except NoSuchElementException:
                    product_names.append("-")
                try:
                    rating = d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@class='a-icon-alt']").text.strip()
                    ratings.append(rating)
                    num_ratings.append(d.find_element_by_xpath("//div[@id='averageCustomerReviews_feature_div']//span[@id='acrCustomerReviewText']").text.strip())
                except NoSuchElementException:
                    ratings.append("-")
                    num_ratings.append("-")            
                try:
                    prices.append(d.find_element_by_id("priceblock_ourprice").text.strip())
                except NoSuchElementException:
                    try:
                        prices.append(d.find_element_by_id("priceblock_dealprice").text.strip())
                    except NoSuchElementException:
                        try:
                            prices.append(d.find_element_by_id("priceblock_saleprice").text.strip())
                        except NoSuchElementException:
                            prices.append("-")
                try:
                    expected_delivery.append(d.find_element_by_id("ddmDeliveryMessage").text.strip())
                except NoSuchElementException:
                    expected_delivery.append("-")
                try:
                    avl = d.find_element_by_id("availability").text.strip()
                    if len(avl) > 0:
                        availabilities.append(avl)
                    else:
                        availabilities.append("-")
                except NoSuchElementException:
                    availabilities.append("-")
                prod_urls.append(d.current_url)
                try:
                    other_details.append(d.find_element_by_id('feature-bullets').text)
                except NoSuchElementException:
                    other_details.append("-")
                try:
                    returns.append(d.find_element_by_xpath("//div[@id='RETURNS_POLICY']//a").text)
                except NoSuchElementException:
                    returns.append("-")

            if next_tab:
                d.close()
                d.switch_to.window(main_window)
            else:
                d.back()



        try:
            next_page_url = d.find_element_by_xpath("//li[@class='a-last']//a").get_attribute('href')
            d.quit()
            d = launch_drive(next_page_url)

        except NoSuchElementException:
            break
    data=list(zip(brands,product_names,ratings,num_ratings,prices,expected_delivery,availabilities,returns,other_details))
    df = pd.DataFrame(data,columns=['Brand Name', "Name of the Product", "Rating", "No. of Ratings", "Price", "Expected Delivery", "Availability","Returns/Exchange", "Other Details"])
    if save_as_csv:
        df.to_csv(product+"_details.csv", sep=',')
        print(f"Save the data as a csv file {product}_details.csv")
    return d, df

In [21]:
d, Data = fetch_product_details_amazon(keyword,3,True)
d.quit()

Showing 1-48 of over 1,000 results for "Guitars"
Save the data as a csv file Guitars_details.csv


In [22]:
Data

,Brand Name,Name of the Product,Rating,No. of Ratings,Price,Expected Delivery,Availability,Returns/Exchange,Other Details
0,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",,"9,242 ratings","₹ 2,369.00",,In stock.,7 Days Replacement,"About this item\nBlack Glossy Finish, Number o..."
1,Visit the JUAREZ Store,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...",,"3,163 ratings","₹ 2,499.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,"About this item\nBlack glossy finish, number o..."
2,Visit the Intern Store,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",,"5,259 ratings","₹ 2,490.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,About this item\nGreat looks with an innovativ...
3,Visit the JUAREZ Store,JUAREZ JRZ38C Right Handed Acoustic Guitar (Su...,,"3,347 ratings","₹ 2,499.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,About this item\nNumber of Frets: 18\nAcoustic...
4,Visit the JUAREZ Store,"Juarez Acoustic Guitar, 38 Inch Cutaway with P...",,"2,129 ratings","₹ 2,369.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,"About this item\nRed Sunburst Glossy Finish, N..."
5,Visit the JUAREZ Store,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",,779 ratings,"₹ 2,499.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,"About this item\nJumbo Design, 38 Inch Acousti..."
6,Visit the Intern Store,Intern 38-inch Cutaway Design Black Acoustic G...,,"1,021 ratings","₹ 1,990.00",,In stock.,7 Days Replacement,About this item\nGreat looks with an innovativ...
7,Brand: HOVNER,Hovner 215 Black F-Cut Rosewood Fretboard Acou...,,203 ratings,"₹ 3,980.00",This item cannot be shipped to your selected d...,Only 1 left in stock.,7 Days Replacement,About this item\n41 inch F- Cut Acoustic Guita...
8,Brand: Giuson,Giuson Venus Black 41 Inch Rosewood Fretboard ...,,135 ratings,"₹ 3,790.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,About this item\nProduct Sold by K-Retail\n› S...
9,Brand: Belear,Belear I-280 Couturier Series 38 Inch Black Sp...,-,-,"₹ 2,399.00",This item cannot be shipped to your selected d...,In stock.,7 Days Replacement,About this item\nPremium 38 Inch single cutawa...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’

In [23]:
### We would be writting a function which would help us to download images
def download_images(folder_name, file_name, img_urls):
    img_urls = img_urls
    DIR = folder_name
    
    if not os.path.exists(DIR):
                os.mkdir(DIR)
    DIR = os.path.join(DIR, file_name)

    if not os.path.exists(DIR):
                os.mkdir(DIR)

    elif os.path.exists(DIR):
        folder = DIR
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

    ###downloading images
    for i ,img in enumerate(img_urls):
        try:        
            r = requests.get(img, stream = True, timeout = 60)
            
            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True
    
                # Open a local file with wb ( write binary ) permission.
                with open(DIR+'\\'+file_name+'_'+str(i)+'.jpg','wb') as f:
                    shutil.copyfileobj(r.raw, f)

                print(f'Image sucessfully Downloaded: {DIR}\\{file_name}_{str(i)}.jpg')
            else:
                print("Image Could not be retreived")
        except Exception as e:
            print("Image Could not be retreived: ", str(e))

In [26]:
def download_google_images(search_keywords = [], num_images = 10):
    for search_key in search_keywords:
        d = launch_drive("https://images.google.com/")
        img_urls = []

        d.find_element_by_xpath("//input[@title='Search']").send_keys(search_key)
        d.find_element_by_xpath("//button[@type='submit']").click()

        while len(img_urls) < num_images:
            for i in range(5):
                d.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                time.sleep(2)
            img_elmns = d.find_elements_by_xpath("//div[@class='isv-r PNCib MSM1fd BUooTd']")
            
            time.sleep(1)
            for img_elmn in img_elmns:
                img_elmn.click()
                time.sleep(5)
                try:
                    img_url = d.find_element_by_xpath("//a[@rlhc='1']/img[@class='n3VNCb']").get_attribute('src')
                except NoSuchElementException:
                    continue
                if 'http' in img_url and 'jpg' in img_url:
                    img_urls.append(d.find_element_by_xpath("//a[@rlhc='1']/img[@class='n3VNCb']").get_attribute('src'))
                if len(img_urls) >= num_images:
                    break
        print(f"Downloading {search_key} images")
        download_images("images", search_key, img_urls)
        img_urls.clear()
        d.quit()

In [28]:
search_keys = ['fruits','cars','Machine Learning']
download_google_images(search_keys,100)

Image sucessfully Downloaded: images\fruits\fruits_0.jpg
Image sucessfully Downloaded: images\fruits\fruits_1.jpg
Image sucessfully Downloaded: images\fruits\fruits_2.jpg
Image sucessfully Downloaded: images\fruits\fruits_3.jpg
Image sucessfully Downloaded: images\fruits\fruits_4.jpg
Image sucessfully Downloaded: images\fruits\fruits_5.jpg
Image sucessfully Downloaded: images\fruits\fruits_6.jpg
Image sucessfully Downloaded: images\fruits\fruits_7.jpg
Image sucessfully Downloaded: images\fruits\fruits_8.jpg
Image sucessfully Downloaded: images\fruits\fruits_9.jpg
Image Could not be retreived
Image sucessfully Downloaded: images\fruits\fruits_11.jpg
Image sucessfully Downloaded: images\fruits\fruits_12.jpg
Image sucessfully Downloaded: images\fruits\fruits_13.jpg
Image sucessfully Downloaded: images\fruits\fruits_14.jpg
Image Could not be retreived
Image sucessfully Downloaded: images\fruits\fruits_16.jpg
Image sucessfully Downloaded: images\fruits\fruits_17.jpg
Image sucessfully Downlo

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=90.0.4430.93)


### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [29]:
def get_smartphone_details_flipkart_searchresult_page1(smart_phone_name, save_as_csv = False):
    
    brands = []
    smartphone_names = []
    colors = []
    rams = []
    storages = []
    prim_cams = []
    secnd_cams = []
    processor_types = []
    display_sizes = []
    resolutions = []
    battery_caps = []
    processor_cores = []
    prices = []
    product_urls = []
    
    d = launch_drive("https://www.flipkart.com")
    main_window = d.current_window_handle
    d.find_element_by_xpath("//button[contains(text(),'✕')]").click()
    d.find_element_by_xpath("//input[@title='Search for products, brands and more']").send_keys(smart_phone_name)
    d.find_element_by_xpath("//button[@type='submit']").click()

    search_results = d.find_elements_by_xpath("//div[@class='_1AtVbE col-12-12']/div[@class='_13oc-S']")

    for result in search_results:
        result.click()
        time.sleep(3)

        all_windows = d.window_handles
        d.switch_to.window(all_windows[1])

        brand = d.find_element_by_xpath("//div[@class='_3GIHBu'][4]/a[@class='_2whKao']").text
        smartphone_name = d.find_element_by_xpath("//h1").text

        d.find_element_by_xpath("//button[contains(text(),'Read More')]").click()
        time.sleep(1)    
        try:
            color = d.find_element_by_xpath("//td[contains(text(),'Color')]/following-sibling::td//li").text
        except NoSuchElementException:
            color = "-"
        try:
            ram = d.find_element_by_xpath("//td[contains(text(),'RAM')]/following-sibling::td//li").text
        except NoSuchElementException:
            ram = "-"

        try:
            storage = d.find_element_by_xpath("//td[contains(text(),'Internal Storage')]/following-sibling::td//li").text
        except NoSuchElementException:
            storage = "-"

        try:
            prim_cam = d.find_element_by_xpath("//li[contains(text(),'MP')]").text
        except NoSuchElementException:
            prim_cam = "-"

        try:
            processor_type = d.find_element_by_xpath("//td[contains(text(),'Processor Type')]/following-sibling::td//li").text
        except NoSuchElementException:
            processor_type = "-"

        try:
            display_size = d.find_element_by_xpath("//td[contains(text(),'Display Size')]/following-sibling::td//li").text
        except NoSuchElementException:
            display_size = "-"

        try:
            resolution = d.find_element_by_xpath("//td[contains(text(),'Resolution')]/following-sibling::td//li").text
        except NoSuchElementException:
            resolution = "-"

        try:
            processor_core = d.find_element_by_xpath("//td[contains(text(),'Processor Core')]/following-sibling::td//li").text
        except NoSuchElementException:
            processor_core = "-"

        try:
            battery_cap = d.find_element_by_xpath("//td[contains(text(),'Battery Capacity')]/following-sibling::td//li").text
        except NoSuchElementException:
            battery_cap = "-"

        try:
            price = d.find_element_by_xpath("//div[@class='_25b18c']/div").text
        except NoSuchElementException:
            price = "-"

        product_url = d.current_url


        brands.append(brand)
        smartphone_names.append(smartphone_name)
        colors.append(color)
        rams.append(ram)
        storages.append(storage)
        prim_cams.append(prim_cam.split('|')[0])
        try:
            secnd_cams.append(prim_cam.split('|')[1])
        except:
            secnd_cams.append('-')
        processor_types.append(processor_type)
        display_sizes.append(display_size)
        resolutions.append(resolution.split('$')[0])
        battery_caps.append(battery_cap)
        processor_cores.append(processor_core)
        prices.append(price)
        product_urls.append(product_url)

        d.close()
        d.switch_to.window(main_window)
    df = pd.DataFrame({'Brand Name':brands,'Smart Phone Name':smartphone_names,'Color':colors,'RAM':rams,'Storage(ROM)':storages,
                       'Primary Camera':prim_cams,'Secondary Camera':secnd_cams,'Display Size':display_sizes,
                      'Display Resolution':resolutions,'Processor':processor_types,'Processor Core':processor_cores,
                       'Battery Capacity':battery_caps,'Price':prices,'Product URL':product_urls})
    
    if save_as_csv:
        df.to_csv(smart_phone_name+'_data.csv', sep=',')
        print(f"Saved the data in {smart_phone_name}_data.csv file!")
    return d,df

In [30]:
d, data = get_smartphone_details_flipkart_searchresult_page1('pixel 4a', save_as_csv = True)
d.quit()
data

Saved the data in pixel 4a_data.csv file!


,Brand Name,Smart Phone Name,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Core,Battery Capacity,Price,Product URL
0,Google Mobiles,"Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)",Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,"₹29,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI Mobiles,"REDMI 9 Power (Blazing Blue, 64 GB) (4 GB RAM)",Blazing Blue,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-blazing...
2,REDMI Mobiles,"REDMI 9 Power (Electric Green, 64 GB) (4 GB RAM)",Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-electri...
3,REDMI Mobiles,"REDMI Note 9 (Aqua Green, 64 GB) (4 GB RAM)",Aqua Green,4 GB,64 GB,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080,MediaTek Helio G85,Octa Core,5020 mAh,"₹10,999",https://www.flipkart.com/redmi-note-9-aqua-gre...
4,REDMI Mobiles,"REDMI 9 Power (Mighty Black, 64 GB) (4 GB RAM)",Mighty Black,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-mighty-...
5,REDMI Mobiles,"REDMI Note 9 (Pebble Grey, 64 GB) (4 GB RAM)",Pebble Grey,4 GB,64 GB,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080,MediaTek Helio G85,Octa Core,5020 mAh,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...
6,REDMI Mobiles,"REDMI Note 9 (Arctic White, 64 GB) (4 GB RAM)",Arctic White,4 GB,64 GB,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080,MediaTek Helio G85,Octa Core,5020 mAh,"₹10,999",https://www.flipkart.com/redmi-note-9-arctic-w...
7,REDMI Mobiles,"REDMI 9 Power (Fiery Red, 64 GB) (4 GB RAM)",Fiery Red,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-fiery-r...
8,REDMI Mobiles,"REDMI 9 Prime (Matte Black, 64 GB) (4 GB RAM)",Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,999",https://www.flipkart.com/redmi-9-prime-matte-b...
9,REDMI Mobiles,"REDMI Note 9 (Pebble Grey, 128 GB) (4 GB RAM)",Pebble Grey,4 GB,128 GB,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080,MediaTek Helio G85,Octa Core,5020 mAh,"₹12,999",https://www.flipkart.com/redmi-note-9-pebble-g...


### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps

In [34]:
def get_long_lat_gmap(city):
    dr = launch_drive("https://www.google.com/maps")

    dr.find_element_by_id("searchboxinput").send_keys(city)
    dr.find_element_by_id("searchbox-searchbutton").click()
    try:
        element = WebDriverWait(dr, 20).until(
            EC.presence_of_element_located((By.XPATH, "//button[@data-value='Directions']"))
        )
    finally: 
        cur_url = dr.current_url

    cur_url = cur_url.split('@')[1].split(',')
    dr.quit()
    return "Longitude: "+cur_url[0],"Latitude: "+cur_url[1]

In [35]:
City = input("Enter item to be searched\n")

Enter item to be searched
nashik


In [38]:
get_long_lat_gmap(City)

TimeoutException: Message: 


### 6.Write a program to scrap details of all the funding deals for Third quarter (i.e. July 20 - September 20) from trak.in

In [40]:
def get_details_trak(quarter, year):
    url = "https://trak.in/"
    q = quarter
    year = str(year)
    months = ['January','February','March','April','May','June','July','August','September','October','November','December']
    dates = []
    start_ups = []
    verticals = []
    sub_verticals =[]
    cities = []
    investors = [] 
    invest_types = []
    amts = []

    print(f"Looking for Quarter {q} details in {year}...")

    if q==1:
        months_window = months[0:3]
    elif q==2:
        months_window = months[3:6]
    elif q==3:
        months_window = months[6:9]
    elif q==4:
        months_window = months[9:]



    d = launch_drive(url)
    d.find_element_by_xpath("//div[@class='container']//a[contains(text(),'Funding Deals')]").click()
    time.sleep(3)
    for month in months_window:
        month = month+", "+year

        print(f'Scrapping details for the month {month}...')
        while True:
            
            try:
                date_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-2']")
                if len(date_elemns) < 1:
                    print(f'Details for {month} not found!')
                    break                 
                start_up_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-3']")
                vertical_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-4']")
                sub_vertical_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-5']")
                city_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-6']")
                investors_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-7']")
                invest_type_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-8']")
                amt_elemns = d.find_elements_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//div[@class='dataTables_scrollBody']//td[@class='column-9']/strong")
            except NoSuchElementException:
                print(f'Details for {month} not found!')
                break

            for date_elemn,start_up_elemn,vertical_elemn,sub_vertical_elemn,city_elemn,investors_elemn,invest_type_elemn,amt_elemn in zip(date_elemns,start_up_elemns,vertical_elemns,sub_vertical_elemns,city_elemns,investors_elemns,invest_type_elemns,amt_elemns):
                dates.append(date_elemn.text)
                start_ups.append(start_up_elemn.text)
                verticals.append(vertical_elemn.text)
                sub_verticals.append(sub_vertical_elemn.text)
                cities.append(city_elemn.text)
                investors.append(investors_elemn.text)
                invest_types.append(invest_type_elemn.text)
                amts.append(amt_elemn.text)

            try:
                d.find_element_by_xpath("//h2[contains(text(),'"+month+"')]/following-sibling::div[1]//a[@class='paginate_button next']").click()
                time.sleep(2)
            except (NoSuchElementException,ElementClickInterceptedException):
                break
    return d, pd.DataFrame({'Date':dates, "Startup Name": start_ups, "Industry/Vertical":verticals, "Sub-Vertical":sub_verticals, "City/Location":cities, "Investors' Name":investors, "Investment type":invest_types, "Amount(In USD)":amts})


In [41]:
d, df = get_details_trak(3,2020)
d.quit()
df

Looking for Quarter 3 details in 2020...
Scrapping details for the month July, 2020...
Scrapping details for the month August, 2020...
Scrapping details for the month September, 2020...


,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investors' Name,Investment type,Amount(In USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [42]:
d, df = get_details_trak(3,2021)
d.quit()
df

Looking for Quarter 3 details in 2021...
Scrapping details for the month July, 2021...
Details for July, 2021 not found!
Scrapping details for the month August, 2021...
Details for August, 2021 not found!
Scrapping details for the month September, 2021...
Details for September, 2021 not found!


,Date,Startup Name,Industry/Vertical,Sub-Vertical,City/Location,Investors' Name,Investment type,Amount(In USD)


### 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in

In [46]:

def get_top_10(search_keyword):
    url = "https://www.digit.in/"
    products = []
    descs = []
    os = []
    displays = []
    processors = []
    memories = []
    weights = []
    dimensions = []
    graph_processors = []
    stocks = []
    prices = []

    d = launch_drive(url)
    time.sleep(3)
    d.find_element_by_class_name("search").click()
    time.sleep(1)
    d.find_element_by_id("globalPageSearchText").send_keys(search_keyword)
    time.sleep(2)
    d.find_element_by_id("globalPageSearchText").send_keys(Keys.RETURN)
    time.sleep(2)
    d.find_element_by_id("content_top10").click()
    search_results_elemn = d.find_element_by_class_name("searchPage")
    search_results_elemn.click()

    for i in range(1,11):
        products.append(d.find_element_by_xpath("//div[@data-index='"+str(i)+"']").text)
        data = d.find_element_by_xpath("//div[@data-index='"+str(i)+"']/following-sibling::div[@class='Section-center'][1]").text.split('SPECIFICATION')
        descs.append(data[0])
        specs = data[1].split("\n")[1:]
        os.append(specs[0].split(':')[1].strip())
        displays.append(specs[1].split(':')[1].strip())
        processors.append(specs[2].split(':')[1].strip())
        memories.append(specs[3].split(':')[1].strip())
        weights.append(specs[4].split(':')[1].strip())
        dimensions.append(specs[5].split(':')[1].strip())
        graph_processors.append(specs[6].split(':')[1].strip())
        stocks.append(specs[7].split('₹')[0].strip().replace('Price :','NA'))
        prices.append(specs[7].split('₹')[1].strip())

    dataframe = pd.DataFrame({'Product Name':products, 'Description':descs, 'OS':os, 'Display':displays, 'Processor':processors, 'Memory':memories,
                 'Weight':weights, 'Dimension':dimensions, 'Graphical Processors':graph_processors, 'Stock Availability':stocks
                 , 'Price':prices})
    return d, dataframe

In [47]:
keyword=input("Enter the product")

Enter the productGaming laptops


In [48]:
d,df = get_top_10(keyword)
d.quit()
df

,Product Name,Description,OS,Display,Processor,Memory,Weight,Dimension,Graphical Processors,Stock Availability,Price
0,1.\nACER ASPIRE 7 A715-75G-50SA,The Acer Aspire 7 laptop isn't exactly built f...,Windows 10,"15.6"" (1920 x 1080)",Core i5 9th Gen | 2.4 GHz with Turbo Boost Upt...,512 NA/8 GBGB DDR4,2.15,363.4 x 254.5 x 23.25,NVIDIA Geforce GTX 1650,NA,54990
1,2.\nMSI GF63 THIN 9SC-240IN,MSI is one of the few manufacturers that are d...,Windows 10,"15.6"" (1920 x 1080)",Core i5 9th Gen | 2.4 GHz with Turbo Boost Upt...,512 GB NA/8 GBGB DDR4,1.86,359 x 254 x 21.7,NVIDIA Geforce GTX 1650 Max Q,NA,59990
2,3.\nHP PAVILLION 15-BC512TX,The HP Pavillion 15-bc512TX is a slight step d...,Windows 10,"15.6"" (1920 x 1080)",9th Gen Intel Core i5 | NA,512 GB NA/8 GBGB DDR4,2.17,36.5 x 25.7 x 2.5,NVIDIA GeForce GTX 1050,AVAILABLE,73719
3,4.\nACER NITRO 5 AN517-51-53JG,The Acer Nitro 5 is the only laptop in this li...,Windows 10,"17.3"" (1920 x 1080)",Core i5 9th Gen | 2.4 GHz,1 TB NA/8 GBGB DDR4,2.7,403 x 280 x 26.9,NVIDIA Geforce GTX 1050,NA,59990
4,5.\nLENOVO LEGION Y7000 81V4000LIN,Legion by Lenovo is a dedicated gaming brand f...,Windows 10,"15.6"" (1920 x 1080)",9th gen Intel Core i5 | 2.4 Ghz,1 GB NA/8 GBGB DDR4,2.3,36 x 26.7 x 2.6,NVIDIA GEFORCE GTX 1050 (3GB GDDR5),AVAILABLE,53990
5,6.\nASUS TUF GAMING FX505DD-AL185T,ASUS TUF Gaming series is about making their h...,Windows 10,"15.6"" (1920 x 1080)",NA,1 TB NA/8 GBGB DDR4,2.2,26.2 x 36 x 2.6,NVIDIA GeForce GTX 1050,AVAILABLE,57990
6,7.\nHP PAVILION GAMING 15-EC0026AX,The HP Pavillion Gaming 15 series of laptops h...,Windows 10,"15.6"" (1920 x 1080)",NA,1 TB NA/8 GBGB DDR4,1.98,25.7 x 36 x 2.4,NVIDIA GeForce GTX 1050,AVAILABLE,62479
7,8.\nASUS TUF GAMING FX505DY-BQ024T,The ASUS TUF Gaming FX505DY comes very close t...,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 5-3550H | 2.1GHz,512GB NA/8 GBGB DDR4,2.2,26.2 x 36 x 2.7,AMD RADEON RX560X,AVAILABLE,54990
8,9.\nMI NOTEBOOK HORIZON EDITION 14,Xiaomi happens to be the latest company to bri...,Windows 10 Home,"14"" (1920 x 1080)",10th Generation Intel® Core™ i7 | 1.8GHz,512 GB SSD/8GB DDR4,1.35,17.15 x 321.3 x 206.8,NVIDIA® GeForce® MX350,AVAILABLE,59999
9,10.\nASUS VIVOBOOK 14 X412FJ-EK513T,Another slim machine in the list is the ASUS V...,Windows 10 Home,"14"" (1920 X 1080)",Intel 8th Gen Core i5 | NA,512 GB SSD/8GB DDR4,1.5 Kg,322 x 212 x 19.9 mm,NVIDIA GeForce MX230 (2GB vRAM),OUT OF STOCK,55990
